In [ ]:
import os
from os import environ

import pickle
from pyquaternion import Quaternion
import numpy as np

import psycopg2
import psycopg2.sql as sql
import postgis.psycopg
import mobilitydb.psycopg


from utils.data_dirs import data_dirs
from utils.index import index
from utils.unique import unique

In [ ]:
EXPERIMENT = False

In [ ]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
connection = psycopg2.connect(
        dbname=environ.get("AP_DB", "mobilitydb"),
        user=environ.get("AP_USER", "docker"),
        host=environ.get("AP_HOST", "localhost"),
        port=environ.get("AP_PORT", "25432"),
        password=environ.get("AP_PASSWORD", "docker"),
    )
connection.autocommit = True
postgis.psycopg.register(connection)
mobilitydb.psycopg.register(connection)

In [ ]:
cursor = connection.cursor()

In [ ]:
with open(os.path.join(output_dir, 'partition/boston-seaport/ground_truth_annotation.pickle'), 'rb') as f:
    annotations = pickle.load(f)

In [ ]:
len(annotations)

In [ ]:
annotations = [a for a in annotations if a['category'].startswith('vehicle')]

In [ ]:
annotations[0]

In [ ]:
trajectories = {}
for a in annotations:
    oid = a['instance_token']
    if oid not in trajectories:
        trajectories[oid] = []
    trajectories[oid].append(a)

In [ ]:
[*trajectories.keys()][:10]

In [ ]:
# direction vector = (1, 0) -> rotate
# then compare the the direction of the segment.

In [ ]:
for dets in trajectories.values():
    for det in dets:
        det['direction'] = Quaternion(det['rotation']).rotate([1, 0, 0])

In [ ]:
tokens = [*range(len(annotations))]

In [ ]:
clss = [*map(lambda x: x['category'], annotations)]

In [ ]:
txs = [*map(lambda x: x['translation'][0], annotations)]
tys = [*map(lambda x: x['translation'][1], annotations)]
tzs = [*map(lambda x: x['translation'][2], annotations)]

In [ ]:
dxs = [*map(lambda x: x['direction'][0], annotations)]
dys = [*map(lambda x: x['direction'][1], annotations)]

In [ ]:
def execute(query, val=None):
    cursor.execute(query, val)
    return cursor.fetchall()

In [ ]:
point = sql.SQL("UNNEST({fields}) AS point (token, tx, ty, tz, dx, dy)").format(
    fields=sql.SQL(',').join(map(sql.Literal, [tokens, txs, tys, tzs, dxs, dys]))
)

cursor.execute(sql.SQL("""
SELECT *
FROM {point}
""").format(point=point))
len(cursor.fetchall())

In [ ]:
points_in_polygon = sql.SQL("""
SELECT segmenttypes

FROM SegmentPolygon as Polygon
GROUP BY segmenttypes
""").format(point=point)
execute(points_in_polygon)

In [ ]:
points_in_polygon = sql.SQL("""
SELECT token
FROM {point}
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point)
len(execute(points_in_polygon))

In [ ]:
min_polygon = sql.SQL("""
SELECT token, MIN(ST_Area(Polygon.elementPolygon)) as size
FROM {point}
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point)
len(execute(min_polygon))

In [ ]:
len(execute(sql.SQL("""
WITH
MinPolygon AS ({min_polygon})
SELECT token, MIN(elementId) as elementId
FROM {point}
JOIN MinPolygon USING (token)
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
GROUP BY token
""").format(point=point, min_polygon=min_polygon)))

In [ ]:
function_angle = sql.SQL("""
DROP FUNCTION IF EXISTS angle(double precision);
CREATE OR REPLACE FUNCTION angle(a double precision) RETURNS double precision AS
$BODY$
BEGIN
    RETURN ((a::decimal % 360) + 360) % 360;
END
$BODY$
LANGUAGE 'plpgsql';
""")

In [ ]:
_segment_with_direction = sql.SQL("""
SELECT
    *,
    ST_X(endPoint) - ST_X(startPoint) AS _x,
    ST_Y(endPoint) - ST_Y(startPoint) AS _y
FROM Segment
""")
len(execute(_segment_with_direction))

In [ ]:
segment_with_direction = sql.SQL("""
SELECT
    *,
    (_x / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dx,
    (_y / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dy
FROM ({_segment_with_direction}) AS foo
WHERE
    _x <> 0 OR _y <> 0
""").format(_segment_with_direction=_segment_with_direction)
len(execute(segment_with_direction))

In [ ]:
execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT *
FROM SegmentWithDirection
WHERE elementId = '8e3c85d1-6d99-44b0-b34a-28a3895485f1'
""").format(segment_with_direction=segment_with_direction))

In [ ]:
execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT *, ST_X(startPoint), ST_Y(startPoint), ST_X(endPoint), ST_Y(endPoint)
FROM Segment
WHERE elementId = '8e3c85d1-6d99-44b0-b34a-28a3895485f1'
""").format(segment_with_direction=segment_with_direction))

In [ ]:
polygon_segment = sql.SQL("""
SELECT elementId
FROM SegmentPolygon
""").format(segment_with_direction=segment_with_direction)
len(execute(polygon_segment))

In [ ]:

execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT elementId, segmenttypes
FROM SegmentPolygon
WHERE NOT EXISTS (
    SELECT *
    FROM SegmentWithDirection
    WHERE SegmentPolygon.elementId = SegmentWithDirection.elementId
)
""").format(segment_with_direction=segment_with_direction))

In [ ]:
polygon_segment = sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT elementId
FROM SegmentPolygon
JOIN SegmentWithDirection USING (elementId)
GROUP BY elementId
""").format(segment_with_direction=segment_with_direction)
len(execute(polygon_segment))

In [ ]:
min_polygon_id = sql.SQL("""
WITH MinPolygon AS ({min_polygon})
SELECT token, MIN(elementId) as elementId
FROM {point}
JOIN MinPolygon USING (token)
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point, min_polygon=min_polygon)
len(execute(min_polygon_id))

In [ ]:
execute(
    sql.SQL("""
    WITH MinPolygonId as ({min_polygon_id})

    SELECT segmenttypes
    FROM SegmentPolygon
    JOIN MinPolygonId USING (elementId)
    GROUP BY segmenttypes
    """).format(min_polygon_id=min_polygon_id)
)

In [ ]:
point_polygon_segment = sql.SQL("""
WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id})

SELECT token
FROM {point}
JOIN MinPolygonId USING (token)
JOIN SegmentPolygon USING (elementId)
JOIN SegmentWithDirection USING (elementId)
WHERE
    angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) < 90
    OR
    angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) > 270
    OR
    'intersection' = Any(SegmentPolygon.segmenttypes)
GROUP BY token
    
""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id
)
len(execute(point_polygon_segment))

In [ ]:
execute(sql.SQL("""
WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id})

SELECT token, tx, ty, tz, dx, dy, elementId, elementPolygon, segmenttypes
FROM {point}
JOIN MinPolygonId USING (token)
JOIN SegmentPolygon USING (elementId)
WHERE
    NOT EXISTS (
        SELECT *
        FROM SegmentWithDirection
        WHERE SegmentWithDirection.elementId = SegmentPolygon.elementId AND
        (
            angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) < 90
            OR
            angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) > 270
            OR
            'intersection' = Any(SegmentPolygon.segmenttypes)
        )
    )
""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id
))

In [ ]:
segment_map = execute(sql.SQL("""
{function_angle}

WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id}),
PointPolygonSegment AS (
    SELECT
        *,
        ST_Distance(ST_Point(tx, ty), ST_MakeLine(startPoint, endPoint)) AS distance,
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) AS anglediff
    FROM {point}
    JOIN MinPolygonId USING (token)
    JOIN SegmentPolygon USING (elementId)
    JOIN SegmentWithDirection AS sd USING (elementId)
    WHERE
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) < 90
        OR
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) > 270
        OR
        'intersection' = Any(SegmentPolygon.segmenttypes)
),
MinDis as (
    SELECT token, MIN(distance) as mindistance
    FROM PointPolygonSegment
    GROUP BY token
),
MinDisMinAngle as (
    SELECT token, MIN(LEAST(pps.anglediff, 360-pps.anglediff)) as minangle
    FROM PointPolygonSegment AS pps
    JOIN MinDis USING (token)
    WHERE pps.distance = MinDis.mindistance
    GROUP BY token
)

SELECT *
FROM PointPolygonSegment
JOIN MinDis USING (token)
JOIN MinDisMinAngle USING (token)
WHERE PointPolygonSegment.distance = MinDis.mindistance
    AND PointPolygonSegment.anglediff = MinDisMinAngle.minangle


""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id,
    function_angle=function_angle
))
cursor.description

In [ ]:
[
    {
        **annotations[s[0]],
        'segmentid': s[1],
        'polygonid': s[9]
    }
    for s
    in segment_map
]

In [ ]:
segment_trajectory = {}
for s in segment_map:
    annotation = annotations[s[0]]
    segmentid = s[1]
    polygonid = s[9]
    itoken = annotation['instance_token']
    if itoken not in segment_trajectory:
        segment_trajectory[itoken] = []
    segment_trajectory[itoken].append({
        **annotation,
        'segmentid': segmentid,
        'polygonid': polygonid,
    })

In [ ]:
segment_trajectory

In [ ]:
def map_points_and_directions_to_segment(annotations: "list"):
    tokens = [*range(len(annotations))]
    clss = [*map(lambda x: x['category'], annotations)]
    txs = [*map(lambda x: x['translation'][0], annotations)]
    tys = [*map(lambda x: x['translation'][1], annotations)]
    tzs = [*map(lambda x: x['translation'][2], annotations)]
    dxs = [*map(lambda x: x['direction'][0], annotations)]
    dys = [*map(lambda x: x['direction'][1], annotations)]
    
    _point = sql.SQL("UNNEST({fields}) AS _point (token, tx, ty, tz, dx, dy)").format(
        fields=sql.SQL(',').join(map(sql.Literal, [tokens, txs, tys, tzs, dxs, dys]))
    )
    
    out = sql.SQL("""
    DROP FUNCTION IF EXISTS angle(double precision);
    CREATE OR REPLACE FUNCTION angle(a double precision) RETURNS double precision AS
    $BODY$
    BEGIN
        RETURN ((a::decimal % 360) + 360) % 360;
    END
    $BODY$
    LANGUAGE 'plpgsql';

    WITH
    Point AS (SELECT * FROM {_point}),
    _SegmentWithDirection AS (
        SELECT
            *,
            ST_X(endPoint) - ST_X(startPoint) AS _x,
            ST_Y(endPoint) - ST_Y(startPoint) AS _y
        FROM Segment
    ),
    SegmentWithDirection AS (
        SELECT
            *,
            (_x / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dx,
            (_y / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dy
        FROM _SegmentWithDirection
        WHERE
            _x <> 0 OR _y <> 0
    ),
    MinPolygon AS (
        SELECT token, MIN(ST_Area(Polygon.elementPolygon)) as size
        FROM Point AS p
        JOIN SegmentPolygon AS Polygon
            ON ST_Contains(Polygon.elementPolygon, ST_Point(p.tx, p.ty))
            AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
        GROUP BY token
    ),
    MinPolygonId AS (
        SELECT token, MIN(elementId) as elementId
        FROM Point AS p
        JOIN MinPolygon USING (token)
        JOIN SegmentPolygon as Polygon
            ON ST_Contains(Polygon.elementPolygon, ST_Point(p.tx, p.ty))
            AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
            AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
        GROUP BY token
    ),
    PointPolygonSegment AS (
        SELECT
            *,
            ST_Distance(ST_Point(tx, ty), ST_MakeLine(startPoint, endPoint)) AS distance,
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) AS anglediff
        FROM Point AS p
        JOIN MinPolygonId USING (token)
        JOIN SegmentPolygon USING (elementId)
        JOIN SegmentWithDirection AS sd USING (elementId)
        WHERE
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) < 90
            OR
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) > 270
            OR
            'intersection' = Any(SegmentPolygon.segmenttypes)
    ),
    MinDis as (
        SELECT token, MIN(distance) as mindistance
        FROM PointPolygonSegment
        GROUP BY token
    ),
    MinDisMinAngle as (
        SELECT token, MIN(LEAST(pps.anglediff, 360-pps.anglediff)) as minangle
        FROM PointPolygonSegment AS pps
        JOIN MinDis USING (token)
        WHERE pps.distance = MinDis.mindistance
        GROUP BY token
    )

    SELECT token, elementid, segmentid
    FROM PointPolygonSegment
    JOIN MinDis USING (token)
    JOIN MinDisMinAngle USING (token)
    WHERE PointPolygonSegment.distance = MinDis.mindistance
        AND PointPolygonSegment.anglediff = MinDisMinAngle.minangle
    """).format(_point=_point)
    
    result = execute(out)
    return result

In [ ]:
map_points_and_directions_to_segment(annotations)[:60]